# Applying Bayesian multinomial logistic regression to the Iris dataset

First we load the dataset in. Due to the format of our data, we don't need to preprocetrain_test_splitssing tasks such as converting objects to categories. We then split into training and testing sets.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pymc3 as pm
import pandas as pd
import numpy as np
import theano
import theano.tensor as tt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()

X = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
y = pd.Series(data=iris['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

Now we train the Bayesian model. To represent the likelihood we use a categorical distribution. To represent the probability of each class, each class has its own regression equation. Then, the equations as a group have the softmax function applied to them to ensure that the total probability sums to 1.

In [2]:
no_of_classes = len(y_train.unique())

with pm.Model() as bayesian_model:
    # Priors
    # Alpha represents the intercepts of each equation
    alpha = pm.Normal('alpha', mu=0, sd=1, shape=no_of_classes)
    # Beta represents the coefficients of the explanatory variables for each equation
    beta = pm.Normal('beta', mu=0, sd=1, shape=(X_train.shape[1], no_of_classes))
    
    # Likelihood
    equations = alpha + pm.math.dot(X_train, beta)
    probabilities = pm.Deterministic('probabilities', tt.nnet.softmax(equations))
    y = pm.Categorical("y", p=probabilities, observed=y_train)
    
    # Sampling
    step = pm.Metropolis()
    trace = pm.sample(5000, step=step)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta]
>Metropolis: [alpha]
Sampling 4 chains, 0 divergences: 100%|██████████| 22000/22000 [00:02<00:00, 7810.46draws/s]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


Now we preform predicitions using the Bayesian model that we just created.

In [4]:
predictions = trace['alpha'].mean(axis=0) + np.dot(X_test, trace['beta'].mean(axis=0))
predicited_classes = np.argmax(np.exp(predictions).T / np.sum(np.exp(predictions), axis=1), axis=0)

bayesianTestingAccuracy = accuracy_score(y_test, predicited_classes)
print("Bayesian model testing accuracy: " + str(bayesianTestingAccuracy * 100) + "%")

Bayesian model testing accuracy: 100.0%
